In [9]:
import sys
sys.path.append('/home/dbswldms316/FLamby/')

from flamby.datasets.fed_tcga_brca.dataset import TcgaBrcaRaw, FedTcgaBrca
from torch.utils.data import Dataset, DataLoader

mydataset_raw = TcgaBrcaRaw()
mydataset_pooled = FedTcgaBrca(train=True, pooled=True)

from flamby.datasets.fed_tcga_brca import (
    BATCH_SIZE,
    LR,
    NUM_EPOCHS_POOLED,
    Baseline,
    BaselineLoss,
    metric,
    NUM_CLIENTS,
    Optimizer,
)

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        # 여기서 필요한 전처리나 데이터 형태 변환을 수행합니다.
        return sample

import torch
# Instantiation of local train set (and data loader)), baseline loss function, baseline model, default optimizer
for i in [0,1,2,3,4,5]:
    train_dataset = FedTcgaBrca(center=i, train=True, pooled=False)
    test_dataset = FedTcgaBrca(center=i, train=False, pooled=False)
    total_dataset = train_dataset + test_dataset
    #print(len(total_dataset))
    
    age_groups = {}
    for person in total_dataset:
        age = int(person[0][0])
        age_group = int(age - age % 10)
        if age_group not in age_groups:
            age_groups[age_group] = []
        age_groups[age_group].append(person)
    age_groups = dict(sorted(age_groups.items()))
    
    result_dict = {'0 to 30':[], '40 to 60':[], '70 to 100':[]}
    for key in age_groups.keys():
        if 0 < key <=30:
            result_dict['0 to 30']+=(age_groups[key])
        elif 30 < key <= 70:
            result_dict['40 to 60']+=(age_groups[key])
        else:
            result_dict['70 to 100']+=(age_groups[key])
    
    i = len(result_dict['0 to 30'])
    new_train_dataset = CustomDataset(result_dict['0 to 30'][:i//2]) 
    new_test_dataset = CustomDataset(result_dict['0 to 30'][i//2:]) 
    i = len(result_dict['40 to 60'])
    new_train_dataset += CustomDataset(result_dict['40 to 60'][:i//2]) 
    new_test_dataset += CustomDataset(result_dict['40 to 60'][i//2:]) 
    i = len(result_dict['70 to 100'])
    new_train_dataset += CustomDataset(result_dict['70 to 100'][:i//2]) 
    new_test_dataset += CustomDataset(result_dict['70 to 100'][i//2:]) 
    print(len(new_test_dataset))
    
    
    lossfunc = BaselineLoss()
    model = Baseline()
    optimizer = Optimizer(model.parameters(), lr=LR)
    new_train_dataloader = torch.utils.data.DataLoader(new_train_dataset, batch_size=16, shuffle=False)
    
    # Traditional pytorch training loop
    for epoch in range(0, NUM_EPOCHS_POOLED):
        for X,y in new_train_dataloader:
            optimizer.zero_grad()
            outputs = model(X)
            loss = lossfunc(outputs, y)
            loss.backward()
            optimizer.step()
            
    from flamby.utils import evaluate_model_on_tests
    age_groups = {}
    for person in new_test_dataset:
        age = int(person[0][0])
        age_group = age - age % 10
        if age_group not in age_groups:
            age_groups[age_group] = []
        age_groups[age_group].append(person)
    age_groups = dict(sorted(age_groups.items()))
    
    result_dict = {'0 to 30': [], '31 to 70': [], '71 to 100': []}
    for key in age_groups.keys():
        if key <= 30:
            result_dict['0 to 30'] += age_groups[key]
        elif key <= 70:
            result_dict['31 to 70'] += age_groups[key]
        else:
            result_dict['71 to 100'] += age_groups[key]

    print('** test dataset includes age groups like: ',[len(age_groups[i]) for i in age_groups.keys()])
    print('** test dataset after split: ', [len(result_dict[i]) for i in result_dict.keys()])
    
    # 각 나이 범주에 해당하는 데이터셋 생성
    test_dataset_0_to_30 = result_dict['0 to 30']
    test_dataset_40_to_60 = result_dict['31 to 70']
    test_dataset_70_to_100 = result_dict['71 to 100']

    # 각 데이터셋을 DataLoader로 변환
    test_dataloader_0_to_30 = torch.utils.data.DataLoader(test_dataset_0_to_30,  shuffle=False)
    test_dataloader_40_to_60 = torch.utils.data.DataLoader(test_dataset_40_to_60,  shuffle=False)
    test_dataloader_70_to_100 = torch.utils.data.DataLoader(test_dataset_70_to_100,  shuffle=False)
    # evaluate_model_on_tests 함수 호출
    with torch.no_grad():
        dict_cindex_0_to_30 = evaluate_model_on_tests(model, [test_dataloader_0_to_30], metric)
        dict_cindex_40_to_60 = evaluate_model_on_tests(model, [test_dataloader_40_to_60], metric)
        dict_cindex_70_to_100 = evaluate_model_on_tests(model, [test_dataloader_70_to_100], metric)

    # 결과 출력
    print("Results for age group 0 to 39:", dict_cindex_0_to_30)
    print("Results for age group 41 to 79:", dict_cindex_40_to_60)
    print("Results for age group 80 to 100:", dict_cindex_70_to_100)


/home/dbswldms316/FLamby/flamby/datasets/fed_tcga_brca/dataset.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (torch.tensor(x, dtype=self.X_dtype), torch.tensor(y, dtype=self.y_dtype))


156
** test dataset includes age groups like:  [13, 9, 79, 46, 7, 2]
** test dataset after split:  [13, 134, 9]


100%|██████████| 1/1 [00:00<00:00, 745.39it/s]

Results for age group 0 to 39: {'client_test_0': 0.6666666666666666}
Results for age group 41 to 79: {'client_test_0': 0.6539735099337748}
Results for age group 80 to 100: {'client_test_0': 0.5384615384615384}
98



/home/dbswldms316/FLamby/flamby/datasets/fed_tcga_brca/dataset.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (torch.tensor(x, dtype=self.X_dtype), torch.tensor(y, dtype=self.y_dtype))


...

Results for age group 0 to 39: {'client_test_0': 0.6666666666666666}
Results for age group 41 to 79: {'client_test_0': 0.7152542372881356}
Results for age group 80 to 100: {'client_test_0': 1.0}
81



/home/dbswldms316/FLamby/flamby/datasets/fed_tcga_brca/dataset.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (torch.tensor(x, dtype=self.X_dtype), torch.tensor(y, dtype=self.y_dtype))


** test dataset includes age groups like:  [6, 11, 42, 20, 2]
** test dataset after split:  [6, 73, 2]


  0%|          | 0/1 [00:00<?, ?it/s]


ZeroDivisionError: No admissable pairs in the dataset.